In [2]:
import tensorflow as tf


import os

import numpy as np
import pandas as pd
from window_generator import WindowGenerator

from run_automation_utils import Params, init_model

from typing import Dict


In [3]:
def prep_window_generator(station_id):
    df = pd.read_feather(f"../../data/pollution/processed/interpolated/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["pm10"])


In [4]:
# window_generator_814 = prep_window_generator(814)
# window_generator_530 = prep_window_generator(530)
window_generator_538 = prep_window_generator(538)

In [5]:
cnn_lstm_model = tf.keras.models.load_model(
    "..\models\cnn_lstm_7days_history_1days_pred_100_epochs_2lstm-layer_64,64_dropout-0-4"
)

In [6]:
cnn_lstm_model.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

In [7]:
cnn_lstm_model.evaluate(window_generator_538.test, verbose=0)

[311.2147521972656, 13.030845642089844]

In [16]:
window_generator_538.plot(cnn_lstm_model, offset=60, plot_version="train")

In [10]:
window_generator_530 = prep_window_generator(530)

In [11]:
cnn_lstm_model_538 = tf.keras.models.load_model(
    "..\models\cnn_lstm_7days_history_1days_pred_100_epochs_2lstm-layer_64,64_dropout-0-4_1"
)

In [12]:
cnn_lstm_model_538.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

In [13]:
cnn_lstm_model_538.evaluate(window_generator_530.test, verbose=0)

[802.9977416992188, 14.610549926757812]

In [15]:
window_generator_530.plot(cnn_lstm_model_538, offset=50, plot_version="train")